In [2]:
import gc
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn import metrics
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import time

In [3]:
DATA_PATH = "../"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [4]:
data.loc[data['label'] == -1, 'label'] = 1

In [5]:
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0

In [6]:
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_b.csv")
data_test = pd.read_csv(csv_path)

In [6]:
# data.fillna(0, inplace=True)
# data_test.fillna(0, inplace=True)

In [7]:
data_train = data[data['date'] <= 20171021]
data_val = data[data['date'] > 20171021]

In [8]:
del data
gc.collect()

28

In [9]:
X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

In [10]:
del data_train, data_val
gc.collect()

62

In [11]:
train = xgb.DMatrix(X_train, label = y_train)
val = xgb.DMatrix(X_val, label = y_val)
test = xgb.DMatrix(X_test)

In [16]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return  score[0]

In [29]:
param = {
    'max_depth': 6, 
    'eta': 0.05, 
    'gamma':1,
    'scale_pos_weight': 1,      
    'min_child_weight': 3, 
    'subsample': 0.8, 
    'booster': 'gbtree', 
    'colsample_bytree': 0.9,
    'objective': 'binary:logistic',
    'eval_metric':'auc',
    'lambda':3
}


In [30]:
model = xgb.train(
    param, 
    train, 
    200, 
    [(val, 'val')],
    early_stopping_rounds=10)

[0]	val-auc:0.936135
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.95437
[2]	val-auc:0.955109
[3]	val-auc:0.957581
[4]	val-auc:0.960628
[5]	val-auc:0.961434
[6]	val-auc:0.961113
[7]	val-auc:0.961395
[8]	val-auc:0.962154
[9]	val-auc:0.962179
[10]	val-auc:0.962451
[11]	val-auc:0.962383
[12]	val-auc:0.963012
[13]	val-auc:0.962542
[14]	val-auc:0.962704
[15]	val-auc:0.962756
[16]	val-auc:0.963643
[17]	val-auc:0.963732
[18]	val-auc:0.963693
[19]	val-auc:0.96364
[20]	val-auc:0.963828
[21]	val-auc:0.964026
[22]	val-auc:0.964289
[23]	val-auc:0.964364
[24]	val-auc:0.965301
[25]	val-auc:0.965428
[26]	val-auc:0.965905
[27]	val-auc:0.966044
[28]	val-auc:0.966068
[29]	val-auc:0.966164
[30]	val-auc:0.966129
[31]	val-auc:0.966244
[32]	val-auc:0.966191
[33]	val-auc:0.966258
[34]	val-auc:0.966362
[35]	val-auc:0.966422
[36]	val-auc:0.966515
[37]	val-auc:0.96655
[38]	val-auc:0.966774
[39]	val-auc:0.966821
[40]	val-auc:0.966968
[41]	val-auc:0.967027
[42]	val-auc:0.967096
[43]	val-auc

In [31]:
[199]	val-auc:0.977774
print(eval_func(1 - y_train, 1 - model.predict(xgb.DMatrix(X_train))))
print(eval_func(1 - y_val, 1 - model.predict(xgb.DMatrix(X_val))))

0.538118968518
0.421688466112


In [ ]:
0.53
0.418

In [32]:
y_pred = 1 - model.predict(test)
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("submission.csv", index=False)